In [1]:
#This is dev branch
import datetime
import math
import copy
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
import logging

NUM_TIME_SLOTS = 144
NUM_DAYS_IN_DATA = 0

In [2]:
# 24 hours is divided into 144 slots where each slot is 10 mins long
def calculateTimeSlot(time,printValue=True):
    global NUM_TIME_SLOTS
    dateTime = datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    timePart = dateTime.time()
    timeInMinutes = (timePart.hour * 60) + timePart.minute + (timePart.second/60) + 1
    timeSlot = timeInMinutes/10
    roundedTimeSlot = math.ceil(timeSlot)
    if roundedTimeSlot > NUM_TIME_SLOTS:
        roundedTimeSlot -= 1
    if printValue==True:
        print(f"time: {time} timeInMinutes: {timeInMinutes} timeSlot: {roundedTimeSlot}")
    return int(roundedTimeSlot)

def extractDayOfWeek(time):
    dateTime = datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    return dateTime.weekday()


In [3]:
def readMultipleData(path,fileNamePrefix,headerNames,dataTypes):
    global NUM_DAYS_IN_DATA
    filesToExplore = []
    for file in os.listdir(path):
        if file.startswith(fileNamePrefix):
            filesToExplore.append(file)
            # print(f"{file} read")
        else:
            continue

    print(f"{len(filesToExplore)} files read")
    if fileNamePrefix == 'order':
        NUM_DAYS_IN_DATA = len(filesToExplore)
    
    readData = []
    for files in filesToExplore:
        fileRead = pd.read_csv(path + files, sep='\t', names=headerNames,dtype=dataTypes)
        readData.append(fileRead)

    readData = pd.concat(readData, ignore_index=True)
    return readData

In [4]:
# now region Data
regionData = pd.read_csv('./training_data/cluster_map/cluster_map', sep='\t', names=['region_hash', 'region_id'],dtype={'region_hash': 'str', 'region_id': 'int'})
# print(regionData.head())
regionData.to_csv('regionData.csv',index=False)

In [5]:
# read order data
dataTypes = {'order_id':'str', 'driver_id':'str', 'passenger_id':'str', 'start_region_hash':'str', 'dest_region_hash':'str', 'price':'double', 'time':'str'}
orderDataPath = './training_data/order_data/'
orderData = readMultipleData(orderDataPath,'order', ['order_id', 'driver_id', 'passenger_id', 'start_region_hash', 'dest_region_hash', 'price', 'time'], dataTypes)
print("printing order data")
# print(orderData.head())

# read weather data
dataTypes={'time':'str', 'weather':'int', 'temperature':'double', 'PM2.5':'double'}
weatherDataPath = './training_data/weather_data/'
weatherData = readMultipleData(weatherDataPath,'weather', ['time', 'weather', 'temperature', 'PM2.5'], dataTypes)
print("printing weather data")
# print(weatherData.head())

21 files read
printing order data
21 files read
printing weather data


In [6]:
weatherData['time'] = weatherData['time'].apply(calculateTimeSlot,printValue=False)
weatherData = weatherData.drop(['temperature','PM2.5'], axis=1)
print(weatherData.head())

   time  weather
0     1        1
1     1        1
2     2        1
3     2        1
4     3        1


In [7]:
orderData = pd.merge(regionData,orderData, how='left', right_on='start_region_hash', left_on='region_hash')
orderData['time_slot'] = orderData['time'].apply(calculateTimeSlot,printValue=False)
orderData['day_of_week'] = orderData['time'].apply(extractDayOfWeek)
orderData = orderData.drop(['passenger_id', 'dest_region_hash','start_region_hash','price','region_hash'], axis=1)
print(orderData.head())
regionData=None

   region_id                          order_id  \
0          1  1654babc363bc6d0f5d01fc0bafedc1a   
1          1  5ac4ac8d0e6092ea1dc323d367613ffd   
2          1  ce86a6ae5eee7a2ea954323b6c01510b   
3          1  b5e816c08e44565c7ed67a6f6e366708   
4          1  25ed10b13aaa36071deecab3aa374be3   

                          driver_id                 time  time_slot  \
0  cc26812d679c9e55a6bf63eed315e989  2016-01-01 20:49:15        126   
1  f6c760be3cd8521c612657da7788f9dc  2016-01-01 13:04:32         79   
2  360478560b1fd4b3eb757074c91ee709  2016-01-01 21:00:21        127   
3  0359fc335d238c6206703d1d7e3620c8  2016-01-01 19:25:32        117   
4  c03944aff7444c27fd7b04cdd3e80af5  2016-01-01 20:35:36        124   

   day_of_week  
0            4  
1            4  
2            4  
3            4  
4            4  


In [8]:
orderData = pd.merge(weatherData,orderData, how="inner", left_on='time', right_on='time_slot')
print(orderData)
orderData.to_csv('orderData.csv',index=False)

MemoryError: Unable to allocate 2.00 GiB for an array with shape (267815373, 1) and data type int64

In [ ]:
# type(orderData['driver_id'][4])==float

In [ ]:
# read POI Data
poiDataStr = {
    'region_hash':[],
    'poi_class':[]
}
with open('./training_data/poi_data/poi_data','r') as fileToRead:
    for line in fileToRead:
        line = line.strip()
        columns = line.split('\t')
        poiDataStr['region_hash'].append(columns[0])
        remData = columns[1:]
        poiDataStr['poi_class'].append(remData)
        
poiData = pd.DataFrame(poiDataStr,columns=['region_hash','poi_class'])
# print("printing poi data")
# print(poiData.head())
# print("Printing poi data line 1")
# print(f"region_hash: {poiData['region_hash'][0]} poi_class: {poiData['poi_class'][0]}")

In [ ]:
# orderData = pd.merge(orderData,poiData, how="outer",right="region_hash",left="start_region_hash")
# print(orderData)

In [ ]:
# now to calculate gap(i,j) = req(i,j) - supply(i,j)
# req(i,j) is for region i and timeslot j 
# ith region will be from from start_region_hash and jth timeslot will be calculated from time
def getRegionID(regionHash):
    regionID = -1
    for i in range(len(regionData)):
        if regionHash == regionData['region_hash'][i]:
            regionID = regionData['region_id'][i]
    return regionID

In [ ]:
mergedData = None # here

In [ ]:
# drop order_id, driver_id, passenger_id, dest_region_hash
# mergedData = orderData.drop(['order_id', 'passenger_id', 'dest_region_hash'], axis=1)
# print("dropped order_id, passenger_id, dest_region_hash")
#  merge order data with region data on start_region_hash with region_hash
# mergedData = pd.merge(regionData,mergedData, how='left', right_on='start_region_hash', left_on='region_hash')
# print("merged order data and region data based on region")

# mergedData = mergedData.drop(['region_hash','start_region_hash'], axis=1)
# print("dropped region_hash, start_region_hash")
# # reduce time to time slot and update time column
# mergedData['time'] = mergedData['time'].apply(calculateTimeSlot,printValue=False)
# print("reduced time to time slot 1 to 144")
# rename time to time_slot
# mergedData.rename(columns={'time':'time_slot'}, inplace=True)
# # append column for day of week into mergedData
# mergedData['day_of_week'] = orderData['time'].apply(extractDayOfWeek)
# print("appended day_of_week column to data")
# now we have mergedData with region_id, price, time, day_of_week
# print("printing merged data")
# print(mergedData)

# writing to mergedData.csv for quick access
orderData.to_csv('mergedData.csv',index=False)

In [ ]:
# read mergedData.csv
mergedDataCSV = pd.read_csv('mergedData.csv')


In [ ]:
# print(mergedDataCSV)

In [ ]:
# print(mergedDataCSV)
mergedDataCSV['requests'] = 1
groupedMergedDataCSV = mergedDataCSV.groupby(['region_id','time_slot','day_of_week','weather','temperature','PM2.5'])['requests'].agg('sum').reset_index()
mergedData = pd.merge(mergedDataCSV,groupedMergedDataCSV, how='left')
print("printing grouped merged data")
# print(mergedData)


In [ ]:
# # now to get req(i,j) we can do that by counting the number of orders for region i and timeslot j
# def getRequest(i,j): # i is region id and j is timeslot
#     global orderData
#     numberOfIterations = len(orderData)
#     print(f"Number of lines of data: {numberOfIterations}")
#     progressBarInit = tqdm(total=numberOfIterations, desc="Calculating requests", unit=" lines")
#     requests = 0
#     for row in range(len(orderData)):
#         currentRegionID = getRegionID(orderData['start_region_hash'][row])
#         currentTimeSlot = calculateTimeSlot(orderData['time'][row],False)
#         if currentRegionID == i and currentTimeSlot == j:
#             requests += 1
#         progressBarInit.update(1)
#     progressBarInit.close()
#     return requests

# print("Printing request for 1st region and 1st timeslot")
# print(getRequest(1,1))

In [ ]:
# # def dateToIndex(date):
# #     index = 0
    
# #     return index

# def getAllRequestAndSupply(): # need to filter by date 
#     global orderData
#     global regionData
#     global NUM_TIME_SLOTS
#     global NUM_DAYS_IN_DATA
#     numberOfRegions = len(regionData)
#     numberOfIterations = len(orderData)
#     print(f"Number of lines of data: {numberOfIterations}")
#     progressBarInit = tqdm(total=numberOfIterations, desc="Calculating requests", unit=" lines")
#     # 3D requests array requests[i][j][k] is number of requests --> date i ,region j, timeslot k
#     # requests = [[[0 for k in range(NUM_TIME_SLOTS)] for j in range(numberOfRegions)] for i in range(NUM_DAYS_IN_DATA)]
#     # supply = [[[0 for k in range(NUM_TIME_SLOTS)] for j in range(numberOfRegions)] for i in range(NUM_DAYS_IN_DATA)]
#     requests = [[0 for j in range(NUM_TIME_SLOTS)] for i in range(numberOfRegions)]
#     supply = [[0 for j in range(NUM_TIME_SLOTS)] for i in range(numberOfRegions)]
#     for row in range(len(orderData)):
#         currentRegionID = getRegionID(orderData['start_region_hash'][row])
#         currentTimeSlot = calculateTimeSlot(orderData['time'][row],False)
#         # date = orderData['time'][row].split(' ')[0]
        
#         if currentRegionID < 0:
#             print(f"Region not found for {orderData['start_region_hash'][row]}")
#             continue
#             # return (None,None)
#         if currentTimeSlot < 0:
#             print(f"Time slot not found for {orderData['time'][row]}")
#             continue
#             # return (None,None)
#         if currentRegionID > numberOfRegions:
#             print(f"Region id {currentRegionID} is greater than number of regions {numberOfRegions}")
#             continue
#             # return (None,None)
#         if currentTimeSlot > NUM_TIME_SLOTS:
#             print(f"Time slot {currentTimeSlot} is greater than number of time slots {NUM_TIME_SLOTS}")
#             print(f"Time: {orderData['time'][row]}")
#             print(f"Row: {row}")
#             continue
#             # return (None,None)
#         if currentTimeSlot == 0:
#             print(f"Time slot is 0 for {orderData['time'][row]}")
#             print(f"Row: {row}")
#             continue
#             # return (None,None)
#         # requests[currentDate][currentRegionID-1][currentTimeSlot-1] += 1
#         requests[currentRegionID-1][currentTimeSlot-1] += 1
#         if type(orderData['driver_id'][row]) == str:
#             supply[currentRegionID-1][currentTimeSlot-1] += 1
#             # supply[currentDate][currentRegionID-1][currentTimeSlot-1] += 1
#         progressBarInit.update(1)
#     progressBarInit.close()
#     return (requests,supply)

# print("Printing request and supply regions d(i) and timeslots t(j)")
# (request,supply) = getAllRequestAndSupply()
# print(request)
# print(supply) 


In [ ]:
# npRequest = np.array(request)
# npSupply = np.array(supply)
# np.savetxt('request.csv', npRequest, delimiter=',')
# np.savetxt('supply.csv', npSupply, delimiter=',')


In [ ]:
# # now to get req(i,j) we can do that by counting the number of orders for region i and timeslot j
# # concurrentI =0
# # concurrentJ =0
# def getRequestMulti(data, i, j,lowerIndex,upperIndex):
#     # (orderData, i, j,lowerIndex,upperIndex) = arguments
#     numberOfIterations = upperIndex - lowerIndex
#     currentPID = mp.current_process()._identity[0]-1
#     # logging.info(f"process {currentPID}")
#     print(f"Number of lines of data: {numberOfIterations} for process {currentPID}")
#     # progressBarInit = tqdm(total=numberOfIterations, desc=f"Calculating requests {currentPID}", unit=" lines")
#     lowerIndex = lowerIndex[currentPID]
#     upperIndex = upperIndex[currentPID]
#     requests = 0
#     for row in range(lowerIndex,upperIndex):
#         currentRegionID = getRegionID(data['start_region_hash'][row])
#         currentTimeSlot = calculateTimeSlot(data['time'][row],False)
#         if currentRegionID == i and currentTimeSlot == j:
#             requests += 1
#         # progressBarInit.update(1)
#     # progressBarInit.close()
#     return requests

# def getRequestHelper(i,j): # i is region id and j is timeslot
#     global orderData
#     # global concurrentI
#     # global concurrentJ
#     # concurrentJ = j
#     # concurrentI = i
#     numberOfIterations = len(orderData)
#     # logging.basicConfig(level=logging.INFO,filename='worker.log', filemode='w')
#     # console_handler = logging.StreamHandler()
#     # logging.getLogger().addHandler(console_handler)
#     # print(f"Number of lines of data: {numberOfIterations}")
#     # progressBarInit = tqdm(total=numberOfIterations, desc="Calculating requests", unit=" lines")
    
#     numberOfProcessesToRun = mp.cpu_count()
#     print(f"CPUs: {numberOfProcessesToRun}")
#     multiProcessingPool = mp.Pool(numberOfProcessesToRun)
#     upperIndex = []
#     lowerIndex = []
#     for i in range(numberOfProcessesToRun):
#         lowerval = i*numberOfIterations//numberOfProcessesToRun
#         upperVal = (i+1)*numberOfIterations//numberOfProcessesToRun
#         lowerIndex.append(lowerval)
#         upperIndex.append(upperVal)
#     print("Here")
#     # argumentsToPass = (orderData, i, j,lowerIndex, upperIndex)
#     requests = multiProcessingPool.starmap(getRequestMulti, [(orderData, i, j,lowerIndex, upperIndex)])
#     requests = sum(requests)
#     multiProcessingPool.close()
#     multiProcessingPool.join()
#     return requests



# print("Printing request for 1st region and 1st timeslot")
# print(getRequestHelper(1,1))